In [3]:
from datetime import date

import pandas as pd
import numpy as np
import holidays
import yaml

Para inicializar la dimension creamos un dataframe donde vamos a añadir las fechas y demas campos                   

In [4]:
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='1/1/2009', freq='D')
})
dim_fecha.head()

,date
0,2005-01-01
1,2005-01-02
2,2005-01-03
3,2005-01-04
4,2005-01-05


vamos a añadir algunas columnas como lo son el año, mes, el dia, el dia de la semana y en que quarto del año

In [5]:
dim_fecha["year"] = dim_fecha["date"].dt.year
dim_fecha["month"] = dim_fecha["date"].dt.month
dim_fecha["day"] = dim_fecha["date"].dt.day
dim_fecha["weekday"] = dim_fecha["date"].dt.weekday
dim_fecha["quarter"] = dim_fecha["date"].dt.quarter

dim_fecha.head()

,date,year,month,day,weekday,quarter
0,2005-01-01,2005,1,1,5,1
1,2005-01-02,2005,1,2,6,1
2,2005-01-03,2005,1,3,0,1
3,2005-01-04,2005,1,4,1,1
4,2005-01-05,2005,1,5,2,1


ahora vamos a añadir la fecha en formato string para los dias, meses    

In [6]:
dim_fecha["day_of_year"] = dim_fecha["date"].dt.day_of_year
dim_fecha["day_of_month"] = dim_fecha["date"].dt.days_in_month
dim_fecha["month_str"] = dim_fecha["date"].dt.month_name() # run locale -a en unix 
dim_fecha["day_str"] = dim_fecha["date"].dt.day_name() # locale = 'es_CO.UTF8'
dim_fecha["date_str"] = dim_fecha["date"].dt.strftime("%d/%m/%Y")
dim_fecha.head()

,date,year,month,day,weekday,quarter,day_of_year,day_of_month,month_str,day_str,date_str
0,2005-01-01,2005,1,1,5,1,1,31,January,Saturday,01/01/2005
1,2005-01-02,2005,1,2,6,1,2,31,January,Sunday,02/01/2005
2,2005-01-03,2005,1,3,0,1,3,31,January,Monday,03/01/2005
3,2005-01-04,2005,1,4,1,1,4,31,January,Tuesday,04/01/2005
4,2005-01-05,2005,1,5,2,1,5,31,January,Wednesday,05/01/2005


# holidays and weekend

In [7]:
co_holidays = holidays.CO(language="es")
dim_fecha["is_Holiday"] = dim_fecha["date"].apply(lambda x:  x in co_holidays)
dim_fecha["holiday"] = dim_fecha["date"].apply(lambda x: co_holidays.get(x))
dim_fecha["saved"] = date.today()
dim_fecha["weekend"] = dim_fecha["weekday"].apply(lambda x: x>4)
dim_fecha.head()



,date,year,month,day,weekday,quarter,day_of_year,day_of_month,month_str,day_str,date_str,is_Holiday,holiday,saved,weekend
0,2005-01-01,2005,1,1,5,1,1,31,January,Saturday,01/01/2005,True,Año Nuevo,2024-09-17,True
1,2005-01-02,2005,1,2,6,1,2,31,January,Sunday,02/01/2005,False,None,2024-09-17,True
2,2005-01-03,2005,1,3,0,1,3,31,January,Monday,03/01/2005,False,None,2024-09-17,False
3,2005-01-04,2005,1,4,1,1,4,31,January,Tuesday,04/01/2005,False,None,2024-09-17,False
4,2005-01-05,2005,1,5,2,1,5,31,January,Wednesday,05/01/2005,False,None,2024-09-17,False


In [8]:
from sqlalchemy import create_engine

with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

In [9]:
dim_fecha.to_sql('dim_fecha', etl_conn, if_exists='replace',index_label='key_dim_fecha')

462